<a href="https://colab.research.google.com/github/2000siddharth/web_scraping_classification/blob/master/boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from prettytable import PrettyTable
%matplotlib inline

In [0]:
df = pd.read_csv('combined_youtube4.csv')

In [0]:
def cleaner(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", 'can not', phrase)
  
  # general
    phrase = re.sub(r"n\'t"," not", phrase)
    phrase = re.sub(r"\'re'"," are", phrase)
    phrase = re.sub(r"\'s"," is", phrase)
    phrase = re.sub(r"\'ll"," will", phrase)
    phrase = re.sub(r"\'d"," would", phrase)
    phrase = re.sub(r"\'t"," not", phrase)
    phrase = re.sub(r"\'ve"," have", phrase)
    phrase = re.sub(r"\'m"," am", phrase)
    
    return phrase

In [0]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
len(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [0]:
cleaned_description = []

for sentance in tqdm(df['description'].values):
    sentance = str(sentance)
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = cleaner(sentance)
    sentance = re.sub(r'[?|!|\'|"|#|+]', r'', sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    cleaned_description.append(sentance.strip())

 34%|███▎      | 3466/10275 [00:01<00:02, 2754.48it/s]/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|██████████| 10275/10275 [00:03<00:00, 3092.75it/s]


In [0]:
cleaned_title = []

for sentance in tqdm(df['title'].values):
    sentance = str(sentance)
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = cleaner(sentance)
    sentance = re.sub(r'[?|!|\'|"|#|+]', r'', sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    cleaned_title.append(sentance.strip())

100%|██████████| 10275/10275 [00:03<00:00, 3351.40it/s]


In [0]:
df['title'] = cleaned_title

df['description'] = cleaned_description
df.head()

,video_id,title,description,category
0,https://www.youtube.com/watch?v=J8XJjkA5NuQ,dr p j abdul kalam lecture series iit bombay,techfest iit bombay presents dr p j abdul kala...,tutorial
1,https://www.youtube.com/watch?v=C31hcftHBIk,lecture laa ke nachhatar gill full video song ...,angel records presents new punjabi song lectur...,tutorial
2,https://www.youtube.com/watch?v=Xn7KWR9EOGQ,basics stock market beginners lecture ca racha...,get lectures new website opportunity learn con...,tutorial
3,https://www.youtube.com/watch?v=FPaByTt1Yws,musique classique pour la lecture de mozart ch...,musique classique pour la lecture de mozart ch...,tutorial
4,https://www.youtube.com/watch?v=ViyG77Bca4I,pte retell lecture january february predicted ...,practice session covers pte retell lecture rea...,tutorial


In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

labels = LabelEncoder()
df['categoryid'] = labels.fit_transform(df['category'])

X = df['title']
y = df['categoryid']

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

for train_index, test_index in sss.split(X, y):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_Train, X_test = X[train_index], X[test_index]
    y_Train, y_test = y[train_index], y[test_index] 

from sklearn.model_selection import train_test_split
X_train, X_cross, y_train, y_cross = train_test_split(X_Train, y_Train, random_state=0, stratify=y_Train, test_size=0.1)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
count_vect.fit(X_train)
X_train_counts = count_vect.transform(X_train)
X_cross_counts = count_vect.transform(X_cross)
X_test_counts = count_vect.transform(X_test)

In [0]:
from pprint import pprint

**Ada Boost Classifier**

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
abc_0 = AdaBoostClassifier(random_state = 8)

print('Parameters currently in use:\n')
pprint(abc_0.get_params())

Parameters currently in use:

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': 8}


In [0]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1]
 }

pre_gs_inst = RandomizedSearchCV(AdaBoostClassifier(), param_distributions = param_dist,
 cv=3,
 n_iter = 10,
 n_jobs=-1)

pre_gs_inst.fit(X_train_counts, y_train)

KeyboardInterrupt: ignored

In [0]:
pre_gs_inst.best_params_

In [0]:
best_ada = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
best_ada.fit(X_train_counts, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [0]:
ada_pred = best_ada.predict(X_test_counts)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, best_ada.predict(X_train_counts)))

The training accuracy is: 
0.8549608002162746


In [0]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, ada_pred))

The test accuracy is: 
0.8579075425790754


In [0]:
# Classification report
print("Classification report")
print(classification_report(y_test, ada_pred, target_names=df['category'].unique()))

In [0]:
# Confusion Matrix
print("Confusion Matrix")
print(confusion_matrix(y_test, ada_pred))

**XG BOOST CLASSIFIER**

In [0]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [0]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [0]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print( "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    

In [0]:
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1,
 gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27)

In [0]:
xgb1.fit(X_train_counts, y_train)
xgb1_pred = xgb1.predict(X_test_counts)

# Classification report
print("Classification report")
print(classification_report(y_test, xgb1_pred, target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_test, xgb1_pred))

In [0]:
print("accuracy score")
print(accuracy_score(y_test, xgb1_pred))

In [0]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=3)
gsearch1.fit(X_train_counts, y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [0]:
gsearch1.best_params_


In [0]:
gsearch1.best_score_

In [0]:
param_test3 = {
 'gamma':[i/10.0 for i in range(4,7)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test3, scoring='accuracy',n_jobs=-1,iid=False, cv=2)
gsearch1.fit(X_train_counts, y_train)
gsearch1.best_params_, gsearch1.best_score_

In [0]:
xgb2 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=3, min_child_weight=1,
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27)

In [0]:
xgb2.fit(X_train_counts, y_train)
xgb2_pred = xgb2.predict(X_test_counts)

# Classification report
print("Classification report")
print(classification_report(y_test, xgb2_pred, target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_test, xgb2_pred))

In [0]:
param_test4 = {
  'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test4, scoring='accuracy',n_jobs=-1,iid=False, cv=2)
gsearch1.fit(X_train_counts, y_train)
gsearch1.best_params_, gsearch1.best_score_

In [0]:
param_test7 = {
 'reg_alpha':[0, 0.005, 0.01, 0.05]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test7, scoring='accuracy',n_jobs=-1,iid=False, cv=3)
gsearch1.fit(X_train_counts, y_train)
gsearch1.best_params_, gsearch1.best_score_

In [0]:
xgb3 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=3, min_child_weight=1, reg_alpha=0.01, 
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27)

In [0]:
xgb3.fit(X_train_counts, y_train)
xgb3_pred = xgb3.predict(X_test_counts)

# Classification report
print("Classification report")
print(classification_report(y_test, xgb3_pred, target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_test, xgb3_pred))

In [0]:
param_test7 = {
 'learning_rate':[0.01, 0.1, 0.05, 0.3, 0.5]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=3, 
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test7, scoring='accuracy',n_jobs=-1,iid=False, cv=3)
gsearch1.fit(X_train_counts, y_train)
gsearch1.best_params_, gsearch1.best_score_

In [0]:
xgb4 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=3, min_child_weight=1, reg_alpha=0.01, 
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27)

In [0]:
xgb4.fit(X_train_counts, y_train)
xgb4_pred = xgb4.predict(X_test_counts)

# Classification report
print("Classification report")
print(classification_report(y_test, xgb4_pred, target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_test, xgb4_pred))

In [0]:
print("accuracy score")
print(accuracy_score(y_test, xgb4_pred))

accuracy score
0.883698296836983


In [0]:
# Classification report
print("Classification report")
print(classification_report(y_train, xgb4.predict(X_train_counts), target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_train, xgb4.predict(X_train_counts)))

Classification report
              precision    recall  f1-score   support

    tutorial       0.86      0.98      0.92       754
     science       0.81      0.85      0.83       744
        vlog       0.99      0.96      0.98       748
        food       0.99      0.98      0.98       679
     fashion       0.98      0.98      0.98       748
      travel       0.98      0.96      0.97       783
       music       0.99      0.96      0.97       690
      movies       0.97      0.96      0.97       735
     history       0.84      0.78      0.81       799
         art       0.97      0.94      0.95       718

    accuracy                           0.93      7398
   macro avg       0.94      0.94      0.94      7398
weighted avg       0.94      0.93      0.93      7398

accuracy score
0.9344417410110841


**Performance Training**

In [0]:
# plot learning curve
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

# fit model no training data
model = XGBClassifier(learning_rate =0.1, n_estimators=500, max_depth=3, min_child_weight=1, reg_alpha=0.01, 
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27)
eval_set = [(X_train_counts, y_train), (X_test_counts, y_test)]
model.fit(X_train_counts, y_train, eval_metric=["merror", "mlogloss"], eval_set=eval_set, verbose=True)
# make predictions for test data
y_pred = model.predict(X_test_counts)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['merror'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['mlogloss'], label='Train')
ax.plot(x_axis, results['validation_1']['mlogloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['merror'], label='Train')
ax.plot(x_axis, results['validation_1']['merror'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()

In [0]:
# early stopping
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# fit model no training data
model = XGBClassifier(learning_rate =0.1, n_estimators=300, max_depth=3, min_child_weight=1, reg_alpha=0.01, 
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27)
eval_set = [(X_test_counts, y_test)]
model.fit(X_train_counts, y_train, early_stopping_rounds=10, eval_metric="merror", eval_set=eval_set, verbose=True)
# make predictions for test data
y_pred = model.predict(X_test_counts)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [0]:
# Classification report
print("Classification report")
print(classification_report(y_train, model.predict(X_train_counts), target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_train, model.predict(X_train_counts)))

Classification report
              precision    recall  f1-score   support

    tutorial       0.84      0.97      0.90       754
     science       0.78      0.87      0.82       744
        vlog       0.99      0.96      0.97       748
        food       0.99      0.97      0.98       679
     fashion       0.97      0.98      0.98       748
      travel       0.98      0.95      0.97       783
       music       0.99      0.96      0.97       690
      movies       0.97      0.96      0.97       735
     history       0.85      0.72      0.78       799
         art       0.96      0.94      0.95       718

    accuracy                           0.93      7398
   macro avg       0.93      0.93      0.93      7398
weighted avg       0.93      0.93      0.93      7398

accuracy score
0.9271424709380913


In [0]:
# Classification report
print("Classification report")
print(classification_report(y_test,y_pred, target_names=df['category'].unique()))

print("accuracy score")
print(accuracy_score(y_test,y_pred))

Classification report
              precision    recall  f1-score   support

    tutorial       0.78      0.95      0.86       210
     science       0.66      0.72      0.69       206
        vlog       0.98      0.93      0.96       208
        food       0.98      0.97      0.97       188
     fashion       0.97      0.97      0.97       208
      travel       0.98      0.93      0.95       218
       music       0.95      0.97      0.96       192
      movies       0.97      0.95      0.96       204
     history       0.72      0.59      0.65       222
         art       0.92      0.92      0.92       199

    accuracy                           0.89      2055
   macro avg       0.89      0.89      0.89      2055
weighted avg       0.89      0.89      0.89      2055

accuracy score
0.8871046228710462


In [0]:
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [0]:
stemmer = PorterStemmer()

def porter_stem(sentence):
    # Porter stemming 
    stemmed_sequence = [stemmer.stem(word) for word in tokenizer.tokenize(sentence)
                        if word not in stopwords.words('english')]
    return ' '.join(stemmed_sequence)

stemmed_train = [porter_stem(post) for post in X_train]
stemmed_test = [porter_stem(post) for post in X_test]

In [0]:
count_vect.fit(stemmed_train)
X_train_stem = count_vect.transform(stemmed_train)

X_test_stem = count_vect.transform(stemmed_test)

In [0]:
from time import time

# Read training and test data (from baseline code)
def read_data(name):
    text, targets = [], []

    with open(''.join([IN_FOLDER, '{}.csv']).format(name)) as f:
        for item in DictReader(f):
            text.append(item['text'])
            targets.append(item['category'])

    return text, targets


In [0]:
from sklearn.metrics import f1_score

In [0]:
t0 = time()

model = XGBClassifier(learning_rate =0.1, n_estimators=300, max_depth=3, min_child_weight=1, reg_alpha=0.01, 
 gamma=0.5, subsample=0.8, colsample_bytree=0.8, objective= 'multi:softmax', nthread=4,n_jobs=-1, scale_pos_weight=1, seed=27) 
model.fit(X_train_stem, y_train)
stem_xgb_prediction = model.predict(X_test_stem)

stem_xgb_macro_f1 = f1_score(y_test, stem_xgb_prediction, average='macro')

print('training score:', f1_score(y_train, model.predict(X_train_stem), average='macro'))

print('testing score:', stem_xgb_macro_f1)

print("done in %0.3fs." % (time() - t0))
print(classification_report(y_test, stem_xgb_prediction))

training score: 0.933008403845245
testing score: 0.8917699866598628
done in 34.080s.
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       210
           1       0.65      0.72      0.69       206
           2       0.98      0.96      0.97       208
           3       0.95      0.97      0.96       188
           4       0.97      0.97      0.97       208
           5       0.98      0.93      0.95       218
           6       0.97      0.96      0.96       192
           7       0.96      0.96      0.96       204
           8       0.69      0.59      0.64       222
           9       0.93      0.92      0.93       199

    accuracy                           0.89      2055
   macro avg       0.89      0.89      0.89      2055
weighted avg       0.89      0.89      0.89      2055



In [0]:
print ('Accuracy score:', accuracy_score(y_test, stem_xgb_prediction))

Accuracy score: 0.8895377128953771


In [0]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=-1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [0]:
category = df['category'].unique()
category

array(['tutorial', 'science', 'vlog', 'food', 'fashion', 'travel',
       'music', 'movies', 'history', 'art'], dtype=object)

In [0]:
category_names = {
    0: 'tutorial',
    1: 'science',
    2: 'vlog',
    3: 'food',
    4: 'fashion',
    5: 'travel',
    6: 'music',
    7: 'movies',
    8: 'history',
    9: 'art'
}

In [0]:
category_code = df['categoryid'].unique()
category_code

array([8, 6, 9, 2, 1, 7, 5, 4, 3, 0])

In [0]:
# Indexes of the test set
index_X_test = X_test.index

# We get them from the original df
df_test = df.loc[index_X_test]

# Add the predictions
df_test['Prediction'] = stem_xgb_prediction

# Clean columns
df_test = df_test[['title', 'category', 'categoryid', 'Prediction']]

# Decode
df_test['Category_Predicted'] = df_test['Prediction']
df_test = df_test.replace({'Category_Predicted':category_names})

# Clean columns again
df_test = df_test[['title', 'category', 'Category_Predicted']]

In [0]:
df_test.head()

,title,category,Category_Predicted
4394,bad hair cast talks making horror movie killer...,movies,fashion
5746,china west meets east metropolitan museum art,art,tutorial
2056,top wearable fashion trends style,fashion,science
7239,day life famous food blogger,food,vlog
7217,bengena pitika assam india best indian food blog,food,vlog


In [0]:
condition = (df_test['category'] != df_test['Category_Predicted'])

df_misclassified = df_test[condition]

df_misclassified.head()

,title,category,Category_Predicted
4394,bad hair cast talks making horror movie killer...,movies,fashion
5746,china west meets east metropolitan museum art,art,tutorial
2056,top wearable fashion trends style,fashion,science
7239,day life famous food blogger,food,vlog
7217,bengena pitika assam india best indian food blog,food,vlog


In [0]:
def output_article(row_article):
    print('Actual Category: %s' %(row_article['category']))
    print('Predicted Category: %s' %(row_article['Category_Predicted']))
    print('-------------------------------------------')
    print('Text: ')
    print('%s' %(row_article['title']))

In [0]:
import random
random.seed(18)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

[1056, 9019, 3433]

In [0]:
output_article(df_misclassified.loc[list_samples[0]])

Actual Category: vlog
Predicted Category: art
-------------------------------------------
Text: 
video blogging gear


In [0]:
output_article(df_misclassified.loc[list_samples[1]])

Actual Category: music
Predicted Category: travel
-------------------------------------------
Text: 
ghost fighter ost intrumental lyrics karaoke


In [0]:
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: travel
Predicted Category: movies
-------------------------------------------
Text: 
travel tourism stocks dive virus woes hk resumes trading


In [0]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [0]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

In [0]:
log_reg = LogisticRegression(penalty='l2', C=0.1, n_jobs=-1, multi_class='ovr')
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=-1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)
svc_clf = SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
voting_clf = VotingClassifier(estimators=[('SVC', svc_clf), ('XGBoost', xgb_clf), ('LogReg', log_reg)], voting='hard')

In [0]:
voting_clf.fit(X_train_stem, y_train)

In [0]:
preds = voting_clf.predict(X_test_stem)
acc = accuracy_score(y_test, preds)

f1 = f1_score(y_test, preds, average='macro')

print ('Accuracy is : ', acc)
print ('F1 Score is :', f1)

Accuracy is :  0.8875912408759125
F1 Score is : 0.8904645724578641


In [0]:
from sklearn.metrics import accuracy_score, log_loss, f1_score

In [0]:
print (confusion_matrix(y_test, preds))

[[202   1   1   1   2   0   0   0   3   0]
 [  4 140   0   2   0   1   1   0  58   0]
 [  6   0 198   0   1   0   0   1   0   2]
 [  1   1   0 182   2   0   0   1   1   0]
 [  3   0   0   1 201   0   0   0   0   3]
 [  5   1   1   0   1 207   0   0   2   1]
 [  5   0   0   0   0   0 184   0   2   1]
 [  4   1   0   0   0   0   0 196   0   3]
 [  9  80   0   1   0   1   2   0 129   0]
 [  5   0   0   0   2   0   1   4   2 185]]


In [0]:
print (classification_report(y_test, preds, target_names=df['category'].unique()))

              precision    recall  f1-score   support

    tutorial       0.83      0.96      0.89       210
     science       0.62      0.68      0.65       206
        vlog       0.99      0.95      0.97       208
        food       0.97      0.97      0.97       188
     fashion       0.96      0.97      0.96       208
      travel       0.99      0.95      0.97       218
       music       0.98      0.96      0.97       192
      movies       0.97      0.96      0.97       204
     history       0.65      0.58      0.62       222
         art       0.95      0.93      0.94       199

    accuracy                           0.89      2055
   macro avg       0.89      0.89      0.89      2055
weighted avg       0.89      0.89      0.89      2055



In [0]:
# Indexes of the test set
index_X_test = X_test.index

# We get them from the original df
df_test = df.loc[index_X_test]

# Add the predictions
df_test['Prediction'] = preds

# Clean columns
df_test = df_test[['title', 'category', 'categoryid', 'Prediction']]

# Decode
df_test['Category_Predicted'] = df_test['Prediction']
df_test = df_test.replace({'Category_Predicted':category_names})

# Clean columns again
df_test = df_test[['title', 'category', 'Category_Predicted']]

In [0]:
condition = (df_test['category'] != df_test['Category_Predicted'])

df_misclassified = df_test[condition]

df_misclassified.head()

,title,category,Category_Predicted
4394,bad hair cast talks making horror movie killer...,movies,fashion
5746,china west meets east metropolitan museum art,art,tutorial
2056,top wearable fashion trends style,fashion,science
7239,day life famous food blogger,food,vlog
7217,bengena pitika assam india best indian food blog,food,vlog


In [0]:
def output_article(row_article):
    print('Actual Category: %s' %(row_article['category']))
    print('Predicted Category: %s' %(row_article['Category_Predicted']))
    print('-------------------------------------------')
    print('Text: ')
    print('%s' %(row_article['title']))

In [0]:
import random
random.seed(18)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

[9617, 5808, 5540]

In [0]:
output_article(df_misclassified.loc[list_samples[0]])

Actual Category: art
Predicted Category: tutorial
-------------------------------------------
Text: 
fun diy projects valentine day


In [0]:
output_article(df_misclassified.loc[list_samples[1]])

Actual Category: art
Predicted Category: tutorial
-------------------------------------------
Text: 
rage master full martial arts movie


In [0]:
1056, 9019, 3433

(1056, 9019, 3433)

In [0]:
output_article(df_misclassified.loc[3433])

Actual Category: travel
Predicted Category: movies
-------------------------------------------
Text: 
travel tourism stocks dive virus woes hk resumes trading


In [0]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [0]:
N = 2
for category, categoryid in sorted(category_names):
  indices = np.argsort(model.coef_[categoryid])
  feature_names = np.array(count_vect.get_feature_names())[indices]
  unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
  bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
  print("# '{}':".format(Product))
  print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
  print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))

TypeError: ignored

In [0]:
df.head()

,video_id,title,description,category,categoryid
0,https://www.youtube.com/watch?v=J8XJjkA5NuQ,dr p j abdul kalam lecture series iit bombay,techfest iit bombay presents dr p j abdul kala...,tutorial,8
1,https://www.youtube.com/watch?v=C31hcftHBIk,lecture laa ke nachhatar gill full video song ...,angel records presents new punjabi song lectur...,tutorial,8
2,https://www.youtube.com/watch?v=Xn7KWR9EOGQ,basics stock market beginners lecture ca racha...,get lectures new website opportunity learn con...,tutorial,8
3,https://www.youtube.com/watch?v=FPaByTt1Yws,musique classique pour la lecture de mozart ch...,musique classique pour la lecture de mozart ch...,tutorial,8
4,https://www.youtube.com/watch?v=ViyG77Bca4I,pte retell lecture january february predicted ...,practice session covers pte retell lecture rea...,tutorial,8


In [0]:
category_code

array([8, 6, 9, 2, 1, 7, 5, 4, 3, 0])